In [1]:
import polars as pl
pl.__version__

'0.19.13'

In [2]:
df = pl.scan_csv("TA.csv", try_parse_dates=True)

In [ ]:
df.limit(0).collect().schema

In [30]:
q = (
    df  .select(['NAME', 'Date', 'Median_SP'])
        .filter(pl.col('NAME') == 'Auckland')
        .drop_nulls()
        .group_by(pl.col('Date').dt.year(), maintain_order=True)
        .agg(
            pl.col('NAME').first().alias("Region"), 
            pl.col('Median_SP').mean().round().alias('Median_Price'), 
            pl.col('Median_SP').count().alias('Observations')
        )
    )
res = q.collect()
res

Date,Region,Median_Price,Observations
i32,str,f64,u32
2023,"""Auckland""",975000.0,3
2022,"""Auckland""",1.10625e6,4
2021,"""Auckland""",1.1125e6,4
2020,"""Auckland""",911043.0,4
2019,"""Auckland""",840000.0,4
2018,"""Auckland""",848750.0,4
2017,"""Auckland""",846875.0,4
2016,"""Auckland""",813000.0,4
2015,"""Auckland""",725919.0,4
